Case study 1.1

In [10]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('item_based_matrix.csv')

# Transpose the dataset to make users rows and items columns
df.set_index('Product_ID', inplace=True)
user_item_matrix = df.T  # Rows = users, Columns = items

# Ensure all data is numeric
user_item_matrix = user_item_matrix.apply(pd.to_numeric, errors='coerce')

# Define active users and target items
active_users = ['User_1', 'User_10', 'User_12']
target_items = ['4K TV', 'Action Camera']

# Cosine Similarity Function
def cosine_similarity(matrix, user1, user2):
    ratings_user1 = matrix.loc[user1].values.astype(float)
    ratings_user2 = matrix.loc[user2].values.astype(float)
    dot_product = np.dot(ratings_user1, ratings_user2)
    norm_user1 = np.sqrt(np.sum(ratings_user1**2))
    norm_user2 = np.sqrt(np.sum(ratings_user2**2))
    if norm_user1 == 0 or norm_user2 == 0:
        return 0
    return dot_product / (norm_user1 * norm_user2)

# Compute Cosine Similarity for all users
similarity_scores = {}
for active_user in active_users:
    similarity_scores[active_user] = {}
    for other_user in user_item_matrix.index:
        if active_user != other_user:
            similarity_scores[active_user][other_user] = cosine_similarity(user_item_matrix, active_user, other_user)

# Top 20% Closest Users
def top_20_percent(similarities):
    sorted_users = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    top_20 = int(len(sorted_users) * 0.2)
    return [user for user, _ in sorted_users[:top_20]]

top_users = {active_user: top_20_percent(similarity_scores[active_user]) for active_user in active_users}

# Discount Factor and Discounted Similarity
def discount_factor(matrix, active_user, other_user, threshold=10):
    common_items = np.sum(~matrix.loc[[active_user, other_user]].isna().all())
    return 1 - (common_items / threshold)

discounted_similarity_scores = {}
for active_user in active_users:
    discounted_similarity_scores[active_user] = {}
    for other_user in user_item_matrix.index:
        if active_user != other_user:
            df_value = discount_factor(user_item_matrix, active_user, other_user)
            discounted_similarity_scores[active_user][other_user] = similarity_scores[active_user][other_user] * df_value

# Top 20% Closest Users with Discounted Similarity
discounted_top_users = {active_user: top_20_percent(discounted_similarity_scores[active_user]) for active_user in active_users}

# Predict Ratings Function
def predict_rating(matrix, active_user, item, top_users, similarity_scores):
    numerator = 0
    denominator = 0
    for user in top_users:
        rating = matrix.loc[user, item]
        if pd.notna(rating):
            sim = similarity_scores[active_user][user]
            numerator += sim * rating
            denominator += abs(sim)
    return numerator / denominator if denominator != 0 else np.nan

# Predict Ratings for Both Cosine and Discounted Similarity
predicted_ratings = {}
discounted_predicted_ratings = {}
for active_user in active_users:
    predicted_ratings[active_user] = {}
    discounted_predicted_ratings[active_user] = {}
    for item in target_items:
        predicted_ratings[active_user][item] = predict_rating(user_item_matrix, active_user, item, top_users[active_user], similarity_scores)
        discounted_predicted_ratings[active_user][item] = predict_rating(user_item_matrix, active_user, item, discounted_top_users[active_user], discounted_similarity_scores)

# 1.1.7 Compare Top 20% Users
print("\nComparison of Top 20% Users:")
for active_user in active_users:
    print(f"Active User: {active_user}")
    print(f"  Top Users (Cosine Similarity): {top_users[active_user]}")
    print(f"  Top Users (Discounted Similarity): {discounted_top_users[active_user]}")

# 1.1.8 Compare Predicted Ratings
print("\nComparison of Predicted Ratings:")
for active_user in active_users:
    print(f"Active User: {active_user}")
    for item in target_items:
        print(f"  Item: {item}")
        print(f"    Original Prediction: {predicted_ratings[active_user][item]:.2f}")
        print(f"    Discounted Prediction: {discounted_predicted_ratings[active_user][item]:.2f}")



Comparison of Top 20% Users:
Active User: User_1
  Top Users (Cosine Similarity): ['User_3', 'User_37', 'User_34', 'User_42', 'User_18', 'User_17', 'User_20', 'User_41', 'User_24']
  Top Users (Discounted Similarity): ['User_16', 'User_25', 'User_19', 'User_22', 'User_11', 'User_46', 'User_6', 'User_28', 'User_39']
Active User: User_10
  Top Users (Cosine Similarity): ['User_11', 'User_8', 'User_27', 'User_13', 'User_12', 'User_20', 'User_45', 'User_32', 'User_38']
  Top Users (Discounted Similarity): ['User_15', 'User_49', 'User_23', 'User_26', 'User_17', 'User_28', 'User_39', 'User_25', 'User_44']
Active User: User_12
  Top Users (Cosine Similarity): ['User_27', 'User_8', 'User_24', 'User_13', 'User_32', 'User_33', 'User_16', 'User_38', 'User_10']
  Top Users (Discounted Similarity): ['User_14', 'User_25', 'User_49', 'User_23', 'User_4', 'User_47', 'User_9', 'User_17', 'User_15']

Comparison of Predicted Ratings:
Active User: User_1
  Item: 4K TV
    Original Prediction: 0.34
    Di

Case study 1.2

In [12]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('item_based_matrix.csv')

# Transpose the dataset to make users rows and items columns
df.set_index('Product_ID', inplace=True)
user_item_matrix = df.T  # Rows = users, Columns = items

# Ensure all data is numeric
user_item_matrix = user_item_matrix.apply(pd.to_numeric, errors='coerce')

# Define active users and target items
active_users = ['User_1', 'User_10', 'User_12']
target_items = ['4K TV', 'Action Camera']

# 1.2.1 Mean-Centered Cosine Similarity Function
def mean_centered_cosine_similarity(matrix, user1, user2):
    mean_user1 = matrix.loc[user1].mean()
    mean_user2 = matrix.loc[user2].mean()
    ratings_user1 = (matrix.loc[user1] - mean_user1).fillna(0).values
    ratings_user2 = (matrix.loc[user2] - mean_user2).fillna(0).values
    dot_product = np.dot(ratings_user1, ratings_user2)
    norm_user1 = np.sqrt(np.sum(ratings_user1**2))
    norm_user2 = np.sqrt(np.sum(ratings_user2**2))
    if norm_user1 == 0 or norm_user2 == 0:
        return 0
    return dot_product / (norm_user1 * norm_user2)

# Compute Mean-Centered Cosine Similarity for all users
mean_centered_similarity_scores = {}
for active_user in active_users:
    mean_centered_similarity_scores[active_user] = {}
    for other_user in user_item_matrix.index:
        if active_user != other_user:
            mean_centered_similarity_scores[active_user][other_user] = mean_centered_cosine_similarity(user_item_matrix, active_user, other_user)

# 1.2.2 Top 20% Closest Users
def top_20_percent(similarities):
    sorted_users = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    top_20 = int(len(sorted_users) * 0.2)
    return [user for user, _ in sorted_users[:top_20]]

top_users_mean_centered = {active_user: top_20_percent(mean_centered_similarity_scores[active_user]) for active_user in active_users}

# 1.2.3 Predict Ratings
def predict_rating_mean_centered(matrix, active_user, item, top_users, similarity_scores):
    mean_active_user = matrix.loc[active_user].mean()
    numerator = 0
    denominator = 0
    for user in top_users:
        mean_user = matrix.loc[user].mean()
        rating = matrix.loc[user, item]
        if pd.notna(rating):
            sim = similarity_scores[active_user][user]
            numerator += sim * (rating - mean_user)
            denominator += abs(sim)
    if denominator == 0:
        return np.nan
    return mean_active_user + (numerator / denominator)

predicted_ratings_mean_centered = {}
for active_user in active_users:
    predicted_ratings_mean_centered[active_user] = {}
    for item in target_items:
        predicted_ratings_mean_centered[active_user][item] = predict_rating_mean_centered(user_item_matrix, active_user, item, top_users_mean_centered[active_user], mean_centered_similarity_scores)

# 1.2.4 Discount Factor and Discounted Similarity
def discount_factor(matrix, active_user, other_user, threshold=10):
    common_items = np.sum(~matrix.loc[[active_user, other_user]].isna().all())
    return 1 - (common_items / threshold)

discounted_similarity_mean_centered = {}
for active_user in active_users:
    discounted_similarity_mean_centered[active_user] = {}
    for other_user in user_item_matrix.index:
        if active_user != other_user:
            df_value = discount_factor(user_item_matrix, active_user, other_user)
            discounted_similarity_mean_centered[active_user][other_user] = mean_centered_similarity_scores[active_user][other_user] * df_value

# 1.2.5 Top 20% Closest Users with Discounted Similarity
discounted_top_users_mean_centered = {active_user: top_20_percent(discounted_similarity_mean_centered[active_user]) for active_user in active_users}

# 1.2.6 Predict Ratings Using Discounted Similarity
predicted_ratings_discounted_mean_centered = {}
for active_user in active_users:
    predicted_ratings_discounted_mean_centered[active_user] = {}
    for item in target_items:
        predicted_ratings_discounted_mean_centered[active_user][item] = predict_rating_mean_centered(user_item_matrix, active_user, item, discounted_top_users_mean_centered[active_user], discounted_similarity_mean_centered)

# 1.2.7 and 1.2.8 Comparison Outputs
print("\nComparison of Top 20% Users:")
for active_user in active_users:
    print(f"Active User: {active_user}")
    print(f"  Top Users (Mean-Centered Cosine Similarity): {top_users_mean_centered[active_user]}")
    print(f"  Top Users (Discounted Similarity): {discounted_top_users_mean_centered[active_user]}")

print("\nComparison of Predicted Ratings:")
for active_user in active_users:
    print(f"Active User: {active_user}")
    for item in target_items:
        print(f"  Item: {item}")
        print(f"    Original Prediction: {predicted_ratings_mean_centered[active_user][item]:.2f}")
        print(f"    Discounted Prediction: {predicted_ratings_discounted_mean_centered[active_user][item]:.2f}")



Comparison of Top 20% Users:
Active User: User_1
  Top Users (Mean-Centered Cosine Similarity): ['User_37', 'User_3', 'User_17', 'User_34', 'User_9', 'User_41', 'User_50', 'User_20', 'User_48']
  Top Users (Discounted Similarity): ['User_46', 'User_11', 'User_16', 'User_40', 'User_22', 'User_33', 'User_19', 'User_12', 'User_8']
Active User: User_10
  Top Users (Mean-Centered Cosine Similarity): ['User_11', 'User_13', 'User_8', 'User_27', 'User_12', 'User_45', 'User_20', 'User_32', 'User_38']
  Top Users (Discounted Similarity): ['User_15', 'User_26', 'User_23', 'User_44', 'User_17', 'User_49', 'User_39', 'User_7', 'User_28']
Active User: User_12
  Top Users (Mean-Centered Cosine Similarity): ['User_27', 'User_13', 'User_16', 'User_10', 'User_8', 'User_32', 'User_24', 'User_38', 'User_33']
  Top Users (Discounted Similarity): ['User_4', 'User_47', 'User_14', 'User_23', 'User_3', 'User_1', 'User_49', 'User_43', 'User_44']

Comparison of Predicted Ratings:
Active User: User_1
  Item: 4K 

Case study 1.3

In [14]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('item_based_matrix.csv')

# Transpose the dataset to make users rows and items columns
df.set_index('Product_ID', inplace=True)
user_item_matrix = df.T  # Rows = users, Columns = items

# Ensure all data is numeric
user_item_matrix = user_item_matrix.apply(pd.to_numeric, errors='coerce')

# Define active users and target items
active_users = ['User_1', 'User_10', 'User_12']
target_items = ['4K TV', 'Action Camera']

# 1.3.1 Compute Pearson Correlation Coefficient (PCC)
def pcc_similarity(matrix, user1, user2):
    mean_user1 = matrix.loc[user1].mean()
    mean_user2 = matrix.loc[user2].mean()
    ratings_user1 = matrix.loc[user1] - mean_user1
    ratings_user2 = matrix.loc[user2] - mean_user2
    common_items = matrix.loc[[user1, user2]].dropna(axis=1)
    if len(common_items.columns) == 0:
        return 0
    ratings_user1 = common_items.loc[user1] - mean_user1
    ratings_user2 = common_items.loc[user2] - mean_user2
    numerator = np.dot(ratings_user1, ratings_user2)
    denominator = np.sqrt(np.sum(ratings_user1**2)) * np.sqrt(np.sum(ratings_user2**2))
    return numerator / denominator if denominator != 0 else 0

# Compute PCC for all users
pcc_similarity_scores = {}
for active_user in active_users:
    pcc_similarity_scores[active_user] = {}
    for other_user in user_item_matrix.index:
        if active_user != other_user:
            pcc_similarity_scores[active_user][other_user] = pcc_similarity(user_item_matrix, active_user, other_user)

# 1.3.2 Top 20% Closest Users
def top_20_percent(similarities):
    sorted_users = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    top_20 = int(len(sorted_users) * 0.2)
    return [user for user, _ in sorted_users[:top_20]]

top_users_pcc = {active_user: top_20_percent(pcc_similarity_scores[active_user]) for active_user in active_users}

# 1.3.3 Predict Ratings
def predict_rating_pcc(matrix, active_user, item, top_users, similarity_scores):
    mean_active_user = matrix.loc[active_user].mean()
    numerator = 0
    denominator = 0
    for user in top_users:
        mean_user = matrix.loc[user].mean()
        rating = matrix.loc[user, item]
        if pd.notna(rating):
            sim = similarity_scores[active_user][user]
            numerator += sim * (rating - mean_user)
            denominator += abs(sim)
    if denominator == 0:
        return np.nan
    return mean_active_user + (numerator / denominator)

predicted_ratings_pcc = {}
for active_user in active_users:
    predicted_ratings_pcc[active_user] = {}
    for item in target_items:
        predicted_ratings_pcc[active_user][item] = predict_rating_pcc(user_item_matrix, active_user, item, top_users_pcc[active_user], pcc_similarity_scores)

# 1.3.4 Discount Factor and Discounted Similarity
def discount_factor(matrix, active_user, other_user, threshold=10):
    common_items = np.sum(~matrix.loc[[active_user, other_user]].isna().all())
    return 1 - (common_items / threshold)

discounted_similarity_pcc = {}
for active_user in active_users:
    discounted_similarity_pcc[active_user] = {}
    for other_user in user_item_matrix.index:
        if active_user != other_user:
            df_value = discount_factor(user_item_matrix, active_user, other_user)
            discounted_similarity_pcc[active_user][other_user] = pcc_similarity_scores[active_user][other_user] * df_value

# 1.3.5 Top 20% Closest Users with Discounted Similarity
discounted_top_users_pcc = {active_user: top_20_percent(discounted_similarity_pcc[active_user]) for active_user in active_users}

# 1.3.6 Predict Ratings Using Discounted Similarity
predicted_ratings_discounted_pcc = {}
for active_user in active_users:
    predicted_ratings_discounted_pcc[active_user] = {}
    for item in target_items:
        predicted_ratings_discounted_pcc[active_user][item] = predict_rating_pcc(user_item_matrix, active_user, item, discounted_top_users_pcc[active_user], discounted_similarity_pcc)

# 1.3.7 and 1.3.8 Comparison Outputs
print("\nComparison of Top 20% Users:")
for active_user in active_users:
    print(f"Active User: {active_user}")
    print(f"  Top Users (PCC): {top_users_pcc[active_user]}")
    print(f"  Top Users (Discounted PCC): {discounted_top_users_pcc[active_user]}")

print("\nComparison of Predicted Ratings:")
for active_user in active_users:
    print(f"Active User: {active_user}")
    for item in target_items:
        print(f"  Item: {item}")
        print(f"    Original Prediction: {predicted_ratings_pcc[active_user][item]:.2f}")
        print(f"    Discounted Prediction: {predicted_ratings_discounted_pcc[active_user][item]:.2f}")



Comparison of Top 20% Users:
Active User: User_1
  Top Users (PCC): ['User_37', 'User_3', 'User_17', 'User_34', 'User_9', 'User_41', 'User_50', 'User_20', 'User_48']
  Top Users (Discounted PCC): ['User_46', 'User_11', 'User_16', 'User_40', 'User_22', 'User_33', 'User_19', 'User_12', 'User_8']
Active User: User_10
  Top Users (PCC): ['User_11', 'User_13', 'User_8', 'User_27', 'User_12', 'User_45', 'User_20', 'User_32', 'User_38']
  Top Users (Discounted PCC): ['User_15', 'User_26', 'User_23', 'User_44', 'User_17', 'User_49', 'User_39', 'User_7', 'User_28']
Active User: User_12
  Top Users (PCC): ['User_27', 'User_13', 'User_16', 'User_10', 'User_8', 'User_32', 'User_24', 'User_38', 'User_33']
  Top Users (Discounted PCC): ['User_4', 'User_47', 'User_14', 'User_23', 'User_3', 'User_1', 'User_49', 'User_43', 'User_44']

Comparison of Predicted Ratings:
Active User: User_1
  Item: 4K TV
    Original Prediction: 1.18
    Discounted Prediction: 3.81
  Item: Action Camera
    Original Predi

Case study 2.1

In [29]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('item_based_matrix.csv')

# Set Product_ID as index
df.set_index('Product_ID', inplace=True)

# Define the item-user matrix (rows = items, columns = users)
item_user_matrix = df.copy()  # Rows = items, Columns = users

# Print available items for verification
print("Available items in the dataset:", item_user_matrix.index.tolist())

# Define target items and ensure they match dataset naming
target_items = ['4K TV', 'Action Camera']

# Step 1: Simulate Missing Values
# Select users to simulate missing values for target items
simulated_missing_users = {'4K TV': ['User_1', 'User_5'], 'Action Camera': ['User_8', 'User_12']}
for item, users in simulated_missing_users.items():
    for user in users:
        item_user_matrix.loc[item, user] = np.nan  # Simulate missing values

# 2.1.1 Compute Cosine Similarity (No Mean-Centering)
def cosine_similarity(matrix, item1, item2):
    ratings_item1 = matrix.loc[item1].fillna(0).values
    ratings_item2 = matrix.loc[item2].fillna(0).values
    dot_product = np.dot(ratings_item1, ratings_item2)
    norm_item1 = np.sqrt(np.sum(ratings_item1**2))
    norm_item2 = np.sqrt(np.sum(ratings_item2**2))
    return dot_product / (norm_item1 * norm_item2) if norm_item1 != 0 and norm_item2 != 0 else 0

# Compute similarity for each target item
cosine_similarity_scores = {}
for target_item in target_items:
    cosine_similarity_scores[target_item] = {
        other_item: cosine_similarity(item_user_matrix, target_item, other_item)
        for other_item in item_user_matrix.index if target_item != other_item  # Exclude the target item itself
    }

# 2.1.2 Top 25% Closest Items
def top_percent(similarities, percent):
    sorted_items = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    top_n = int(len(sorted_items) * percent)
    return [item for item, _ in sorted_items[:top_n]]

top_items_cosine = {target_item: top_percent(cosine_similarity_scores[target_item], 0.25) for target_item in target_items}

# 2.1.3 Predict Missing Ratings
def predict_rating(matrix, target_item, user, top_items, similarity_scores):
    numerator = 0
    denominator = 0
    for item in top_items:
        rating = matrix.loc[item, user]
        if pd.notna(rating):
            sim = similarity_scores[item]
            numerator += sim * rating
            denominator += abs(sim)
    return numerator / denominator if denominator != 0 else np.nan

predicted_ratings_cosine = {}
for target_item in target_items:
    predicted_ratings_cosine[target_item] = {}
    for user in simulated_missing_users[target_item]:
        prediction = predict_rating(item_user_matrix, target_item, user, top_items_cosine[target_item], cosine_similarity_scores[target_item])
        predicted_ratings_cosine[target_item][user] = prediction

# 2.1.4 Compute Discount Factor (DF) and Discounted Similarity
def discount_factor(matrix, target_item, other_item, threshold=10):
    common_users = np.sum(~matrix.loc[[target_item, other_item]].isna().all(axis=0))
    return 1 - (common_users / threshold)

discounted_similarity_scores = {}
for target_item in target_items:
    discounted_similarity_scores[target_item] = {
        other_item: cosine_similarity_scores[target_item][other_item] * discount_factor(item_user_matrix, target_item, other_item)
        for other_item in item_user_matrix.index if target_item != other_item  # Exclude the target item itself
    }

# 2.1.5 Top 20% Closest Items (Discounted Similarity)
top_items_discounted = {target_item: top_percent(discounted_similarity_scores[target_item], 0.20) for target_item in target_items}

# 2.1.6 Predict Missing Ratings (Discounted Similarity)
predicted_ratings_discounted = {}
for target_item in target_items:
    predicted_ratings_discounted[target_item] = {}
    for user in simulated_missing_users[target_item]:
        prediction = predict_rating(item_user_matrix, target_item, user, top_items_discounted[target_item], discounted_similarity_scores[target_item])
        predicted_ratings_discounted[target_item][user] = prediction

# 2.1.7 and 2.1.8 Comparison Outputs
print("\nComparison of Top Items:")
for target_item in target_items:
    print(f"Target Item: {target_item}")
    print(f"  Top Items (Cosine Similarity): {top_items_cosine[target_item]}")
    print(f"  Top Items (Discounted Similarity): {top_items_discounted[target_item]}")

print("\nComparison of Predicted Ratings:")
for target_item in target_items:
    print(f"Target Item: {target_item}")
    for user in simulated_missing_users[target_item]:
        original_prediction = predicted_ratings_cosine[target_item][user]
        discounted_prediction = predicted_ratings_discounted[target_item][user]
        print(f"  User: {user}")
        print(f"    Original Prediction: {original_prediction:.2f}")
        print(f"    Discounted Prediction: {discounted_prediction:.2f}")








Available items in the dataset: ['4K TV', 'Action Camera', 'Air Fryer', 'Bluetooth Headphones', 'Coffee Maker', 'Electric Kettle', 'Fitness Tracker', 'Gaming Laptop', 'Gaming Monitor', 'Home Router', 'Mechanical Keyboard', 'Office Chair', 'Portable SSD', 'Robot Vacuum', 'Smart Light Bulbs', 'Smart Speaker', 'Smartwatch', 'USB-C Charger', 'VR Headset', 'Wireless Mouse']

Comparison of Top Items:
Target Item: 4K TV
  Top Items (Cosine Similarity): ['Office Chair', 'Coffee Maker', 'Electric Kettle', 'Robot Vacuum']
  Top Items (Discounted Similarity): ['Wireless Mouse', 'Gaming Laptop', 'Bluetooth Headphones']
Target Item: Action Camera
  Top Items (Cosine Similarity): ['Smart Light Bulbs', 'Fitness Tracker', 'VR Headset', 'Portable SSD']
  Top Items (Discounted Similarity): ['Robot Vacuum', 'Electric Kettle', 'Mechanical Keyboard']

Comparison of Predicted Ratings:
Target Item: 4K TV
  User: User_1
    Original Prediction: 2.02
    Discounted Prediction: -4.32
  User: User_5
    Original

Case study 2.2

In [30]:
# 2.2.1 Apply Cosine Similarity with Mean-Centering
def mean_centered_cosine_similarity(matrix, item1, item2):
    mean_item1 = matrix.loc[item1].mean()
    mean_item2 = matrix.loc[item2].mean()

    # Mean-centered ratings
    ratings_item1 = (matrix.loc[item1] - mean_item1).fillna(0).values
    ratings_item2 = (matrix.loc[item2] - mean_item2).fillna(0).values

    # Compute cosine similarity
    dot_product = np.dot(ratings_item1, ratings_item2)
    norm_item1 = np.sqrt(np.sum(ratings_item1**2))
    norm_item2 = np.sqrt(np.sum(ratings_item2**2))
    return dot_product / (norm_item1 * norm_item2) if norm_item1 != 0 and norm_item2 != 0 else 0

# Compute mean-centered cosine similarity for each target item
mean_centered_similarity_scores = {}
for target_item in target_items:
    mean_centered_similarity_scores[target_item] = {
        other_item: mean_centered_cosine_similarity(item_user_matrix, target_item, other_item)
        for other_item in item_user_matrix.index if target_item != other_item
    }

# 2.2.2 Determine Top 20% Closest Items
top_items_mean_centered = {target_item: top_percent(mean_centered_similarity_scores[target_item], 0.20) for target_item in target_items}

# 2.2.3 Predict Missing Ratings Using Mean-Centered Cosine Similarity
predicted_ratings_mean_centered = {}
for target_item in target_items:
    predicted_ratings_mean_centered[target_item] = {}
    for user in simulated_missing_users[target_item]:
        prediction = predict_rating(item_user_matrix, target_item, user, top_items_mean_centered[target_item], mean_centered_similarity_scores[target_item])
        predicted_ratings_mean_centered[target_item][user] = prediction

# 2.2.4 Compute Discount Factor (DF) and Discounted Similarity (DS)
discounted_mean_centered_similarity_scores = {}
for target_item in target_items:
    discounted_mean_centered_similarity_scores[target_item] = {
        other_item: mean_centered_similarity_scores[target_item][other_item] * discount_factor(item_user_matrix, target_item, other_item)
        for other_item in item_user_matrix.index if target_item != other_item
    }

# 2.2.5 Determine Top 20% Closest Items (Discounted Mean-Centered Similarity)
top_items_discounted_mean_centered = {target_item: top_percent(discounted_mean_centered_similarity_scores[target_item], 0.20) for target_item in target_items}

# 2.2.6 Predict Missing Ratings Using Discounted Mean-Centered Similarity
predicted_ratings_discounted_mean_centered = {}
for target_item in target_items:
    predicted_ratings_discounted_mean_centered[target_item] = {}
    for user in simulated_missing_users[target_item]:
        prediction = predict_rating(item_user_matrix, target_item, user, top_items_discounted_mean_centered[target_item], discounted_mean_centered_similarity_scores[target_item])
        predicted_ratings_discounted_mean_centered[target_item][user] = prediction

# 2.2.7 and 2.2.8 Comparison Outputs
print("\nComparison of Top Items:")
for target_item in target_items:
    print(f"Target Item: {target_item}")
    print(f"  Top Items (Mean-Centered Cosine Similarity): {top_items_mean_centered[target_item]}")
    print(f"  Top Items (Discounted Mean-Centered Similarity): {top_items_discounted_mean_centered[target_item]}")

print("\nComparison of Predicted Ratings:")
for target_item in target_items:
    print(f"Target Item: {target_item}")
    for user in simulated_missing_users[target_item]:
        original_prediction = predicted_ratings_mean_centered[target_item][user]
        discounted_prediction = predicted_ratings_discounted_mean_centered[target_item][user]
        print(f"  User: {user}")
        print(f"    Original Prediction: {original_prediction:.2f}")
        print(f"    Discounted Prediction: {discounted_prediction:.2f}")



Comparison of Top Items:
Target Item: 4K TV
  Top Items (Mean-Centered Cosine Similarity): ['Office Chair', 'Robot Vacuum', 'Coffee Maker']
  Top Items (Discounted Mean-Centered Similarity): ['Bluetooth Headphones', 'Wireless Mouse', 'Smart Light Bulbs']
Target Item: Action Camera
  Top Items (Mean-Centered Cosine Similarity): ['Fitness Tracker', 'Smart Light Bulbs', 'VR Headset']
  Top Items (Discounted Mean-Centered Similarity): ['Mechanical Keyboard', 'Electric Kettle', 'Robot Vacuum']

Comparison of Predicted Ratings:
Target Item: 4K TV
  User: User_1
    Original Prediction: 2.23
    Discounted Prediction: 4.35
  User: User_5
    Original Prediction: 2.31
    Discounted Prediction: 3.69
Target Item: Action Camera
  User: User_8
    Original Prediction: 1.61
    Discounted Prediction: 2.21
  User: User_12
    Original Prediction: 3.06
    Discounted Prediction: 2.89


Case study 2.3

In [31]:
# 2.3.1 Apply PCC to Compute Similarity
def pcc_similarity(matrix, item1, item2):
    common_users = matrix.loc[[item1, item2]].dropna(axis=1)
    if len(common_users.columns) < 2:  # Minimum 2 users needed for PCC
        return 0
    ratings_item1 = common_users.loc[item1]
    ratings_item2 = common_users.loc[item2]
    return np.corrcoef(ratings_item1, ratings_item2)[0, 1]

# Compute PCC similarity for each target item
pcc_similarity_scores = {}
for target_item in target_items:
    pcc_similarity_scores[target_item] = {
        other_item: pcc_similarity(item_user_matrix, target_item, other_item)
        for other_item in item_user_matrix.index if target_item != other_item
    }

# 2.3.2 Determine Top 20% Closest Items
top_items_pcc = {target_item: top_percent(pcc_similarity_scores[target_item], 0.20) for target_item in target_items}

# 2.3.3 Predict Missing Ratings Using PCC Similarity
predicted_ratings_pcc = {}
for target_item in target_items:
    predicted_ratings_pcc[target_item] = {}
    for user in simulated_missing_users[target_item]:
        prediction = predict_rating(item_user_matrix, target_item, user, top_items_pcc[target_item], pcc_similarity_scores[target_item])
        predicted_ratings_pcc[target_item][user] = prediction

# 2.3.4 Compute Discount Factor (DF) and Discounted Similarity (DS)
discounted_pcc_similarity_scores = {}
for target_item in target_items:
    discounted_pcc_similarity_scores[target_item] = {
        other_item: pcc_similarity_scores[target_item][other_item] * discount_factor(item_user_matrix, target_item, other_item)
        for other_item in item_user_matrix.index if target_item != other_item
    }

# 2.3.5 Determine Top 20% Closest Items (Discounted PCC Similarity)
top_items_discounted_pcc = {target_item: top_percent(discounted_pcc_similarity_scores[target_item], 0.20) for target_item in target_items}

# 2.3.6 Predict Missing Ratings Using Discounted PCC Similarity
predicted_ratings_discounted_pcc = {}
for target_item in target_items:
    predicted_ratings_discounted_pcc[target_item] = {}
    for user in simulated_missing_users[target_item]:
        prediction = predict_rating(item_user_matrix, target_item, user, top_items_discounted_pcc[target_item], discounted_pcc_similarity_scores[target_item])
        predicted_ratings_discounted_pcc[target_item][user] = prediction

# 2.3.7 and 2.3.8 Comparison Outputs
print("\nComparison of Top Items:")
for target_item in target_items:
    print(f"Target Item: {target_item}")
    print(f"  Top Items (PCC Similarity): {top_items_pcc[target_item]}")
    print(f"  Top Items (Discounted PCC Similarity): {top_items_discounted_pcc[target_item]}")

print("\nComparison of Predicted Ratings:")
for target_item in target_items:
    print(f"Target Item: {target_item}")
    for user in simulated_missing_users[target_item]:
        original_prediction = predicted_ratings_pcc[target_item][user]
        discounted_prediction = predicted_ratings_discounted_pcc[target_item][user]
        print(f"  User: {user}")
        print(f"    Original Prediction: {original_prediction:.2f}")
        print(f"    Discounted Prediction: {discounted_prediction:.2f}")



Comparison of Top Items:
Target Item: 4K TV
  Top Items (PCC Similarity): ['Office Chair', 'Coffee Maker', 'Robot Vacuum']
  Top Items (Discounted PCC Similarity): ['Bluetooth Headphones', 'Wireless Mouse', 'Smart Light Bulbs']
Target Item: Action Camera
  Top Items (PCC Similarity): ['Fitness Tracker', 'Smart Light Bulbs', 'VR Headset']
  Top Items (Discounted PCC Similarity): ['Mechanical Keyboard', 'Electric Kettle', 'Robot Vacuum']

Comparison of Predicted Ratings:
Target Item: 4K TV
  User: User_1
    Original Prediction: 2.23
    Discounted Prediction: 4.35
  User: User_5
    Original Prediction: 2.32
    Discounted Prediction: 3.69
Target Item: Action Camera
  User: User_8
    Original Prediction: 1.61
    Discounted Prediction: 2.22
  User: User_12
    Original Prediction: 3.08
    Discounted Prediction: 2.90
